# Symbolic regression via neural network weights

## Create equations and respective datasets

We create random quadratic equations in the form +/- ax^2 +/- bx +/- c; and generate X, y pairs for the said equations using the following routine.

In [5]:
from random_quadratics import RandomQuadratic

In [10]:
lower_bound = -5
upper_bound = 5
round_digits = 1
number_of_equations = 6000

equations = []

for _ in range(number_of_equations):
    equation = RandomQuadratic(lower_bound=lower_bound, upper_bound=upper_bound, round_digits=round_digits)
    equations.append(equation())


In [11]:
from numpy.random import uniform
from generate_datasets import GenerateDatasets
from json import dumps
from os import mkdir, path

X_values = uniform(-1, 1, 5000).astype(dtype="float64", copy=False)
root_dir = "./datasets"
dataset_generator = GenerateDatasets(equations=equations, X_values=X_values)
datesets_dict = dataset_generator.generate_xy_datasets()
# dumps makes the float X values in keys into strings

print(dumps(datesets_dict, indent=4))
GenerateDatasets.write_dataset_to_file(datesets_dict, root_dir=root_dir)

Import the saved X,y pairs for the respective equations to pytorch dataloader.

In [12]:
from equations_dataset import EquationsDataset
from json import load
from torch.utils.data import Dataset, DataLoader, random_split
from torch import set_printoptions

root_dir = "./datasets"
equation_data = EquationsDataset(dataset_file_path=f"{root_dir}/Equation5.json")
#data_loader = DataLoader(equation_data, batch_size=1, shuffle=True)
#for idx, xy_values in enumerate(data_loader):
    #print(f"XY at position {idx} is {xy_values}")

In [13]:
#print(data_loader)
#print(equation_data.x_values)

#train, test = random_split(data_loader, [4000, 1000])

#print(vars(train))
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
#train_loader = DataLoader(train, batch_size=1, shuffle=True)
#test_loader = DataLoader(test, batch_size=1, shuffle=True)
#print(vars(train_loader))
#print(train_loader.X)

train_values_x, train_values_y = equation_data.x_values[:4000], equation_data.y_values[:4000] 
test_values_x, test_values_y = equation_data.x_values[4000:], equation_data.x_values[4000:] 

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train_values_x, train_values_y, test_size=0.33, random_state=42)

train_data = []
for i in range(len(X_train)):
    train_data.append([X_train[i], y_train[i]])



val_data = []
for i in range(len(X_val)):
    val_data.append([X_val[i], y_val[i]])

In [14]:
train_loader = DataLoader(train_data, batch_size=1, shuffle=True)
#for idx, xy_values in enumerate(train_loader):
    #print(f"XY at position {idx} is {xy_values}")

val_loader = DataLoader(val_data, batch_size=1, shuffle=True)
#for idx, xy_values in enumerate(val_loader):
    #print(f"XY at position {idx} is {xy_values}")

## Simple MLP to regression on our tasks

In [15]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

In [23]:
def train_model(train_dataloader:DataLoader, validation_dataloader:DataLoader, epochs, model:nn.Module, optimizer, scheduler, criterion):
    train_losses = []
    validation_losses = []

    #train-validation loop
    for epoch in range(epochs):
        batch_losses = []
        training_loss = 0.0
        #training loop
        for _idx , data in enumerate(train_dataloader):
            inputs, labels = data
            optimizer.zero_grad()
            model.train()
            outputs = model(inputs.float())
            loss = criterion(outputs.float(), labels.float())
            loss.backward()
            batch_losses.append(loss.item())
            optimizer.step()
        training_loss = np.mean(batch_losses)
        train_losses.append(training_loss)
        scheduler.step()

        #validation loop
        with torch.no_grad():
            val_losses = []
            validation_loss = 0.0
            for _idx, data in enumerate(validation_dataloader):
                inputs, labels = data
                model.eval()
                outputs = model(inputs.float())
                loss = criterion(outputs.float(), labels.float())
                val_losses.append(loss.item())
            validation_loss = np.mean(val_losses)
            validation_losses.append(validation_loss)

        print(f"[{epoch+1}] Training loss: {training_loss:.7f}\t Validation loss: {validation_loss:.7f}")
        print(f"\t Label value: {labels.float().item()}\t Predicted Output: {outputs.float().item()}")
    #torch.save(model.state_dict(), MODEL_PATH)
    return model.state_dict()

def eval_model(test_dataloader: DataLoader, model: nn.Module, criterion):
    test_losses = []
    with torch.no_grad():
        for _idx, data in enumerate(test_dataloader):
            inputs, labels = data
            model.eval()
            outputs = model(inputs)
            #print("outputs, ", outputs.shape)
            #rescaled_outputs = inverse_scaler(outputs, method="minmax")
            #print("rescaled_outputs: ",rescaled_outputs.shape)
            loss = criterion(outputs, labels)
            test_losses.append(loss.item())
        test_loss = np.mean(test_losses)
        print(f"Final test loss: {test_loss:.4f}")    
    return test_losses

In [103]:
class Test_MLP(nn.Module):

    def __init__(self, input_size, hidden_size, output_size):
        # call constructor from superclass
        super(Test_MLP, self).__init__()
        # define network layers
        self.input_size = input_size
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, 7)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(7, hidden_size)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        # define forward pass
        output = self.fc1(x)
        output = self.relu1(output)
        output = self.fc2(output)
        output = self.relu2(output)
        output = self.fc3(output)
        output = self.relu3(output)
        output = self.fc4(output)
        return output

In [108]:
model = Test_MLP(1, 8, 1)
print(model)

epochs = 300
optimizer  = optim.Adam(model.parameters(), lr=1e-05)
scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=20, eta_min=1e-05)
criterion = nn.MSELoss()

mf_dict =train_model(train_loader, val_loader, epochs, model, optimizer, scheduler, criterion)
#mf_dict.keys().contains("bias")


Test_MLP(
  (fc1): Linear(in_features=1, out_features=8, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=8, out_features=7, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=7, out_features=8, bias=True)
  (relu3): ReLU()
  (fc4): Linear(in_features=8, out_features=1, bias=True)
)
[1] Training loss: 6.2270619	 Validation loss: 5.8835313
	 Label value: 3.1099154949188232	 Predicted Output: 0.579146683216095
[2] Training loss: 5.6806706	 Validation loss: 5.3024153
	 Label value: 3.674118995666504	 Predicted Output: 0.6968907117843628
[3] Training loss: 5.0531033	 Validation loss: 4.6547504
	 Label value: 2.8657939434051514	 Predicted Output: 0.8297504782676697
[4] Training loss: 4.3681413	 Validation loss: 3.9620186
	 Label value: 3.74953293800354	 Predicted Output: 1.120961308479309
[5] Training loss: 3.6687089	 Validation loss: 3.2836861
	 Label value: 3.5961296558380127	 Predicted Output: 1.2942225933074951
[6] Training loss: 2.9847603	 Validation loss: 2.6178079
	 Lab

[63] Training loss: 0.0521185	 Validation loss: 0.0543151
	 Label value: 3.674793004989624	 Predicted Output: 3.798083782196045
[64] Training loss: 0.0501421	 Validation loss: 0.0523536
	 Label value: 3.3045196533203125	 Predicted Output: 3.239467144012451
[65] Training loss: 0.0483148	 Validation loss: 0.0502709
	 Label value: 0.6492391228675842	 Predicted Output: 0.6790294647216797
[66] Training loss: 0.0465210	 Validation loss: 0.0484103
	 Label value: 2.886937379837036	 Predicted Output: 2.9042389392852783
[67] Training loss: 0.0448168	 Validation loss: 0.0466323
	 Label value: 3.7480459213256836	 Predicted Output: 3.671168565750122
[68] Training loss: 0.0432311	 Validation loss: 0.0449760
	 Label value: 2.9625179767608643	 Predicted Output: 3.00754451751709
[69] Training loss: 0.0416902	 Validation loss: 0.0433824
	 Label value: 3.262395143508911	 Predicted Output: 3.2372517585754395
[70] Training loss: 0.0402159	 Validation loss: 0.0418837
	 Label value: 0.7197855114936829	 Predi

[127] Training loss: 0.0060689	 Validation loss: 0.0059152
	 Label value: 0.8791296482086182	 Predicted Output: 0.8538746237754822
[128] Training loss: 0.0058879	 Validation loss: 0.0057544
	 Label value: 3.4173877239227295	 Predicted Output: 3.6880085468292236
[129] Training loss: 0.0057316	 Validation loss: 0.0055873
	 Label value: 3.5231566429138184	 Predicted Output: 3.5437488555908203
[130] Training loss: 0.0055685	 Validation loss: 0.0054279
	 Label value: 2.3686416149139404	 Predicted Output: 2.356760025024414
[131] Training loss: 0.0054092	 Validation loss: 0.0052734
	 Label value: 3.719970464706421	 Predicted Output: 3.657829999923706
[132] Training loss: 0.0052549	 Validation loss: 0.0051240
	 Label value: 3.7338521480560303	 Predicted Output: 3.651557683944702
[133] Training loss: 0.0051090	 Validation loss: 0.0049804
	 Label value: 2.9556994438171387	 Predicted Output: 2.9380807876586914
[134] Training loss: 0.0049662	 Validation loss: 0.0048377
	 Label value: 2.25688028335

[190] Training loss: 0.0014173	 Validation loss: 0.0014241
	 Label value: 2.710103750228882	 Predicted Output: 2.6971709728240967
[191] Training loss: 0.0013941	 Validation loss: 0.0013942
	 Label value: 3.548072099685669	 Predicted Output: 3.542182207107544
[192] Training loss: 0.0013661	 Validation loss: 0.0013804
	 Label value: 0.1456320732831955	 Predicted Output: 0.1283889263868332
[193] Training loss: 0.0013463	 Validation loss: 0.0013535
	 Label value: 3.6091506481170654	 Predicted Output: 3.612236738204956
[194] Training loss: 0.0013265	 Validation loss: 0.0013291
	 Label value: 2.3003275394439697	 Predicted Output: 2.289621114730835
[195] Training loss: 0.0013024	 Validation loss: 0.0013102
	 Label value: 3.4698431491851807	 Predicted Output: 3.562011241912842
[196] Training loss: 0.0012832	 Validation loss: 0.0012982
	 Label value: 3.636927604675293	 Predicted Output: 3.6411292552948
[197] Training loss: 0.0012646	 Validation loss: 0.0012808
	 Label value: 3.6617400646209717	

[253] Training loss: 0.0006516	 Validation loss: 0.0006758
	 Label value: -0.2130238264799118	 Predicted Output: -0.1972765326499939
[254] Training loss: 0.0006426	 Validation loss: 0.0006726
	 Label value: 2.274501323699951	 Predicted Output: 2.2609832286834717
[255] Training loss: 0.0006395	 Validation loss: 0.0006591
	 Label value: 2.694017171859741	 Predicted Output: 2.67587947845459
[256] Training loss: 0.0006334	 Validation loss: 0.0006602
	 Label value: 0.6747041344642639	 Predicted Output: 0.6412810683250427
[257] Training loss: 0.0006271	 Validation loss: 0.0006678
	 Label value: 3.752633571624756	 Predicted Output: 3.7571399211883545
[258] Training loss: 0.0006265	 Validation loss: 0.0006454
	 Label value: 3.611250162124634	 Predicted Output: 3.594015121459961
[259] Training loss: 0.0006172	 Validation loss: 0.0006416
	 Label value: 3.457643508911133	 Predicted Output: 3.510622978210449
[260] Training loss: 0.0006132	 Validation loss: 0.0006371
	 Label value: 1.61143231391906

In [110]:
networks_weights = []
for keys in mf_dict.keys():
    if "bias" not in keys:
        print(keys)
        print(mf_dict[keys].flatten())
        print(mf_dict[keys].flatten().shape)
        networks_weights.append(mf_dict[keys].flatten().tolist())
print()
#print(networks_weights)
networks_weights = [item for sublist in networks_weights for item in sublist]
print(networks_weights)
print(len(networks_weights))

fc1.weight
tensor([-0.4296,  0.0791, -0.2075,  1.1229, -0.8824, -1.1717,  0.0549,  0.3695])
torch.Size([8])
fc2.weight
tensor([ 0.1273,  0.1462,  0.2124,  0.5475,  0.1835, -0.2722,  0.1423, -0.2814,
        -0.4968,  0.3224, -0.2453, -0.2873, -0.2079, -0.5478, -0.3082,  0.4633,
        -0.2312,  0.1844, -0.0031,  0.1777, -0.3513, -0.2784, -0.3505,  0.1783,
        -0.1864, -0.1478,  0.2134, -0.0352, -0.2567, -0.3392, -0.1517,  0.0682,
        -0.2250,  0.1092,  0.1293, -0.3273, -0.2466, -0.2939,  0.2976, -0.2350,
        -0.4759,  0.6274,  0.2777,  0.2897, -0.4809, -0.1402, -0.3190,  0.2209,
         0.0618, -0.0276,  0.1939,  0.4144,  0.2115, -0.2220,  0.2850, -0.0264])
torch.Size([56])
fc3.weight
tensor([ 0.1614, -0.5984, -0.5499, -0.1632,  0.1903, -0.7242,  0.3013, -0.3459,
         0.7090,  0.1920, -0.0781,  0.1575,  0.2099, -0.5087, -0.4621,  0.7770,
         0.0019, -0.0109,  0.0486,  0.1476, -0.6907,  0.3182, -0.1687, -0.2444,
        -0.3658, -0.1470, -0.3790, -0.3586, -0.1509,

## Use AutoPytorch to search for MLP architectures

In [1]:
#%pip list
#from autoPyTorch.api.tabular_classification import TabularClassificationTask
from autoPyTorch import AutoNetRegression
from autoPyTorch.data_management.data_manager import DataManager


# Note: You can write your own datamanager! Call fit train, valid data (numpy matrices) 
#dm = DataManager()
#dm.generate_regression(num_features=21, num_samples=1500)

# Note: every parameter has a default value, you do not have to specify anything. The given parameter allow a fast test.
#autonet = AutoNetRegression(budget_type='epochs', min_budget=1, max_budget=9, num_iterations=1, log_level='info')

#res = autonet.fit(X_train=X_train, Y_train=y_train, X_valid=X_val, Y_valid=y_val)
print(res)

ValueError: numpy.ndarray size changed, may indicate binary incompatibility. Expected 88 from C header, got 80 from PyObject

In [9]:
len(equations)

5000

In [111]:
equation_data.equation_name

'-1.8*x*x+2*x+3.2'